Coordinate systems

ielu saves its electrode coordinates in Surface RAS (tkrRAS) coordinate system.
https://surfer.nmr.mgh.harvard.edu/fswiki/CoordinateSystems

ielu geometry gives some ways to accesss freesurfer affine transform linked to a subject and their resampled base MRI

One thing that is quite nice is that freesurfer includes freeview which can display multiple volumes and surfaces and their coordinates at the same time. You can even specify a 3D coordinate and have the cursor move there in one of these systems:
the CT might be specified in Column, Row, Slice (CRS) form

RAS stands for Right, Anterior, Superior and refers to the axes and direction of increase

LPI stands for left, posterior, inferior


In [72]:
from __future__ import division, print_function
import ielu.geometry
import numpy as np
from ielu.geometry import apply_affine
from numpy.linalg import inv
from numpy import dot
import sympy

In [1]:
e_rpar_vox = [69.0, 178.0, 106.875] # think this is in voxel coord right parietal point on the CT
# ?origin at LPI (left posterior inf)


In [3]:
# as saved in csv file by ielu
e_rpar_tkras = [57.8501, -44.6678,27.5802]

In [7]:
# in freeview (fv) approx the same point looks like
fv_ras = [58.5, 10.71, 66.30]
fv_tkras = [51.40, 28.68, -45.31]
fv_ct_nas = [73,167,107] # this is close to e_rpar_vox
##
fv_surfaceRAS = [55.62, -46.84, 28.11] # fv_tkras = 55.6, 28.1, -46.8 so surfaceRAS has 2nd/3rd components flipped


Below we will define a point taken from freeview -- all listed while the mri brain volume is loaded and the registered ct from ct_nas.nii is loaded. Thus these coordinates all represent the same point just in different coordinate systems.
I chose a point near the surface of brain/skull in the right parietal region.

In [45]:
pt_ras = 64.94, 11.36, 65.2     # RAS 64.94, 11.36, 65.2
pt_tkreg = 57.85, 27.58, -44.66 # TkReg RAS 57.85, 27.58, -44.66
pt_rawavg = 66,176, 106 # rawavg [66,176, 106] row,col,slice? voxel
pt_brain = 70,100, 83   # brain [70,100, 83]
pt_ct = 66, 176, 106    # ct_nas.nii [66, 176, 106]
pt_surf_ras = 57.85, -44.66, 27.58   # rh.pial SurfaceRAS 57.85, -44.66, 27.58

In [11]:
ctfile = '/Users/clee/subjects/JJE2016/mri/ct_nas.nii.gz' # I think this is the reference

ct_affine_vox2ras_tkr = ielu.geometry.get_vox2rasxfm(ctfile, stem='vox2ras-tkr')
ct_affine_vox2ras = ielu.geometry.get_vox2rasxfm(ctfile, stem='vox2ras')

In [10]:
ct_affine_vox2ras_tkr

array([[  -0.9375,    0.    ,    0.    ,  120.    ],
       [   0.    ,    0.    ,    1.    ,  -78.    ],
       [   0.    ,   -0.9375,    0.    ,  120.    ],
       [   0.    ,    0.    ,    0.    ,    1.    ]])

In [12]:
ct_affine_vox2ras

array([[  -0.9375,    0.    ,   -0.    ,  127.094 ],
       [   0.    ,   -0.9375,   -0.    ,  176.021 ],
       [   0.    ,    0.    ,    1.    ,  -40.376 ],
       [   0.    ,    0.    ,    0.    ,    1.    ]])

In [46]:
ct_ras_tkr2ras = np.dot(ct_affine_vox2ras,inv(ct_affine_vox2ras_tkr))
ct_ras_tkr2ras

array([[  1.   ,   0.   ,   0.   ,   7.094],
       [  0.   ,   0.   ,   1.   ,  56.021],
       [  0.   ,   1.   ,   0.   ,  37.624],
       [  0.   ,   0.   ,   0.   ,   1.   ]])

In [28]:
mri_brain = '/Users/clee/subjects/JJE2016/mri/brain.mgz'
mri_brain_vox2ras_tkr = ielu.geometry.get_vox2rasxfm(mri_brain, stem='vox2ras-tkr')
mri_brain_vox2ras = ielu.geometry.get_vox2rasxfm(mri_brain, stem='vox2ras')

In [15]:
mri_brain_vox2ras_tkr

array([[  -1.,    0.,    0.,  128.],
       [   0.,    0.,    1., -128.],
       [   0.,   -1.,    0.,  128.],
       [   0.,    0.,    0.,    1.]])

In [33]:
mri_brain_vox2ras

array([[  -1.     ,    0.     ,    0.     ,  135.09399],
       [   0.     ,    0.     ,    1.     ,  -71.97899],
       [   0.     ,   -1.     ,    0.     ,  165.62399],
       [   0.     ,    0.     ,    0.     ,    1.     ]])

In [55]:
def homogenize(v):
    """turn a 3D vector/sequence into its equivalnent homogenous coordinate vector
    and return the result as a numpy array"""
    return np.array((v[0], v[1],v[2], 1))

# create a shortcut h for the function
h = homogenize

In [62]:
# Now let's see if our points match up
# w = A v
w = dot(inv(ct_affine_vox2ras), homogenize(pt_ras))
w # note this is close to pt_ct

array([  66.2976,  175.6384,  105.576 ,    1.    ])

In [63]:
np.round(w) # if we round to the nearest integer we get back our voxel coordinate

array([  66.,  176.,  106.,    1.])

In [77]:
# try going from RAS -> surfaceRAS
T = dot(mri_brain_vox2ras_tkr, inv(mri_brain_vox2ras))
y = dot(T,h(pt_ras))
print('pt_ras:', pt_ras)
print('pt_surf_ras:', pt_surf_ras)
print('y:', y)
# does pt_surf_ras = y = T . pt_ras? Not exactly but they are very close
print('are they close?\n h(pt_surf_ras) -  y < 0.01 :', h(pt_surf_ras) -  y < 0.01)

pt_ras: (64.94, 11.36, 65.2)
pt_surf_ras: (57.85, -44.66, 27.58)
y: [ 57.84601 -44.66101  27.57601   1.     ]
are they close?
 h(pt_surf_ras) -  y < 0.01 : [ True  True  True  True]


In [23]:
np.dot(inv(ct_affine_vox2ras_tkr), homogenize(e_rpar_tkras))

array([ 66.29322667,  98.58112   ,  33.3322    ,   1.        ])

In [24]:
np.dot(ct_affine_vox2ras, homogenize(_))

array([ 64.9441,  83.6012,  -7.0438,   1.    ])

In [26]:
np.dot(ct_affine_vox2ras_tkr, homogenize(e_rpar_vox))

array([ 55.3125,  28.875 , -46.875 ,   1.    ])

In [29]:
# A = composition of the transforms
A = np.dot(mri_brain_vox2ras, inv(mri_brain_vox2ras_tkr))

In [31]:
np.dot(A, homogenize(e_rpar_tkras)) # this appears work!

array([ 64.94409,  11.35321,  65.20419,   1.     ])

In [32]:
A

array([[  1.     ,   0.     ,   0.     ,   7.09399],
       [  0.     ,   1.     ,   0.     ,  56.02101],
       [  0.     ,   0.     ,   1.     ,  37.62399],
       [  0.     ,   0.     ,   0.     ,   1.     ]])

In [40]:
from IPython.display import display, Math,Latex

In [42]:
Math(A)

TypeError: Math expects text, not array([[  1.     ,   0.     ,   0.     ,   7.09399],
       [  0.     ,   1.     ,   0.     ,  56.02101],
       [  0.     ,   0.     ,   1.     ,  37.62399],
       [  0.     ,   0.     ,   0.     ,   1.     ]])